In [1]:
import cv2
import numpy as np
import pytesseract
import imutils

In [2]:
img=cv2.imread("9.1 licence_plate.jpg.jpg")
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [3]:
# keskin çizgileri yumuşatmak için filtreliyoruz
# resim çap ve sigma değerleri
filtered=cv2.bilateralFilter(gray,6,250,250) # plaka harici yerleri filtrelemeye çalışıyoruz

d: Her piksel komşuluğunun çapı.<br>
sigmaColor:\sigma  Renk uzayındaki değeri  . Değer ne kadar büyük olursa birbirine uzak olan renkler karışmaya başlayacaktır.<br>
sigmaSpace:\sigma  Koordinat alanındaki değer  . Değeri ne kadar büyük olursa, renklerinin sigmaColor aralığında olması koşuluyla daha fazla piksel birbirine karışacaktır.

In [4]:
# şimdi köşeleri algılayacağız
edge=cv2.Canny(filtered,30,200)

In [5]:
# filter olmadan farkı görmek için
edgeFiltersiz=cv2.Canny(gray,30,200)
cv2.imshow("Filter uygulanmamış edge",edgeFiltersiz)
cv2.imshow("edge",edge)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
# counter yani sınır koordinatlarını bulacağız
contours=cv2.findContours(edge,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [7]:
cnts1=imutils.grab_contours(contours) # uygun counterleri yakala demek
#plaka dikdörtgen şekil içinde olduğundan dikdörtgen şeklinde kapalı bir şey arıyoruz / sıralayıp dikdörtgen var mı diye bakacağız
cnts=sorted(cnts1,key=cv2.contourArea,reverse=True)[:10] # sorted sıralama fonksiyonudur , alana göre sırala diyoruz , ters çeviriyoruz / [:10] 0 dan 10a kadar olan değerler için demek
screen=None # gerçekten kapalı şekil bulabildik mi onun kontrolü için kullanacağız
print("contours",contours)
print("************************************")
print("cnts1",cnts1)
print("************************************")
print("cnts",cnts)

contours ((array([[[454, 300]],

       [[453, 301]],

       [[451, 301]],

       [[450, 302]],

       [[449, 302]],

       [[448, 303]],

       [[447, 303]],

       [[446, 304]],

       [[443, 304]],

       [[442, 305]],

       [[439, 305]],

       [[438, 306]],

       [[431, 306]],

       [[430, 305]],

       [[425, 305]],

       [[424, 304]],

       [[403, 304]],

       [[402, 303]],

       [[377, 303]],

       [[376, 304]],

       [[367, 304]],

       [[366, 305]],

       [[360, 305]],

       [[359, 306]],

       [[352, 306]],

       [[351, 307]],

       [[349, 307]],

       [[348, 308]],

       [[347, 308]],

       [[346, 309]],

       [[344, 309]],

       [[343, 310]],

       [[341, 310]],

       [[340, 311]],

       [[333, 311]],

       [[332, 310]],

       [[330, 310]],

       [[329, 309]],

       [[310, 309]],

       [[309, 310]],

       [[303, 310]],

       [[302, 311]],

       [[281, 311]],

       [[277, 307]],

       [[277, 306]],


In [8]:
# föngüde ilk önce countours lara yaklaşıcaz sonra kapalı şekilleri arayacağız
# yaklaşmak yani düzgün hale getiricez
for c in cnts:
    # deneysel formul kullanacağız
    epsilon=0.018*cv2.arcLength(c,True) # arvLenght demek yay uzunluğu demek , True ile boşluk var mı yok muya bakıyoz burada 
    approx=cv2.approxPolyDP(c,epsilon,True) # approx yani yaklaşım
    if len(approx)==4: # 4 yakın köşe varsa yani 4 değer saklıysadikdörtgendir dedik
        screen=approx
        break
print(screen) # koordinatları tutuyor yani plaka koordinatları bunlar

[[[225 218]]

 [[226 247]]

 [[414 242]]

 [[411 214]]]


In [9]:
# şimdi mask uygulayacağız
mask=np.zeros(gray.shape,np.uint8) # mask ile her yeri siyah yaptım
cv2.imshow("mask",mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
# şimdi plakanın olduğu yeri beyaz yapacağız
new_img=cv2.drawContours(mask,[screen],0,(255,255,255),-1)
cv2.imshow("Plaka beyaz yapılmış mask",mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
# şimdi plaka bölgesindeki yazıyı counter alanına yapıştıracağım
new_img=cv2.bitwise_and(img,img,mask=mask)
cv2.imshow("new_img",new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
# şimdi elimizdeki büyük resmi kırpacağız
# beyaz olan bölgeleri tutacağız
x,y=np.where(mask==255) # mask da 255 olan bölgeyi al dedik
print(np.where(mask==255))
# sol üst köşedeki koordinatı alıcaz
(topX,topY)=(np.min(x),np.min(y))
# sağ alt köşedeki koordinatı alıcaz
(bottomX,bottomY)=(np.max(x),np.max(y))
# şimdi kırpıyoruz
crooped=gray[topX:bottomX+1,topY:bottomY+1] # crooped=kırpılmış , 1 fazla alıyoruz ki son değer de gelsin
cv2.imshow("crooped",crooped)
cv2.waitKey(0)
cv2.destroyAllWindows()

(array([214, 214, 214, ..., 247, 247, 247], dtype=int64), array([388, 389, 390, ..., 242, 243, 244], dtype=int64))


In [13]:
# sıra geldi okuyup yazmaya
text=pytesseract.image_to_string(crooped,lang="eng")
print("Plaka : ",text)

Plaka :  300 SL 1957.



In [14]:
# tüm resimleri tek tek görmek istedim
cv2.imshow("img",img)
cv2.imshow("gray",gray)
cv2.imshow("edge",edge)
cv2.imshow("mask",mask)
cv2.imshow("Plaka beyaz yapılmış mask",mask)
cv2.imshow("new_img",new_img)
cv2.imshow("crooped",crooped)
cv2.waitKey(0)
cv2.destroyAllWindows()